In [1]:
import os
memory = '200g'
pyspark_submit_args = ' --driver-memory ' + memory  +' pyspark-shell' +" --num-executors 15 --executor-cores 8"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import numpy as np
from collections import Counter
from HyperLogLog import Hyperloglog as hll
from HyperLogLog import Countsketch
from HyperLogLog import Data2sketchplus
import time
import pandas as pd
import pickle
import math
import random
import findspark
findspark.init()
from pyspark import SparkContext

sc=SparkContext(master="spark://Master:17077",appName="test")
#sc=SparkContext(appName="test")
sc._conf.set("spark.driver.maxResultSize", '50g')

In [2]:
def TransData(partition):
    level=1
    D2S=Data2sketchplus(width,level)
    D2S.ScanString([item.encode("utf-8") for item in partition])
    Item=[item.encode("utf-8") for item in partition]
    yield [[D2S.fiArray(i) for i in range(level)],D2S.NDVArray(),D2S.CSArray()]
def HatC(n,f1):
    return 1.0-f1/n
def DCL(ndv,i2fi,n,f1):
    hc=HatC(n,f1)
    d1hat=ndv/hc
    gamma=max(0, d1hat*(i2fi-n)/(n^2-n-1))
    return ndv/hc+n*(1-hc)/hc*gamma
def partComputef1(ss):
    sketch=pickle.loads(b.value)
    f1=0
    for s in ss:
        f1t=Computef1(sketch,s)
        f1+=f1t
    yield f1
def Computef1(SKList,F1ID):
    k_mechine=len(SKList)
    ZeroSketch=hll(width)
    IntersetID=[]
    for i in range(k_mechine):
        if i!=F1ID:
            ZeroSketch.update(SKList[i][1])
    MergeSketch=hll(width)
    MergeSketch.update(SKList[F1ID][0][0])
    f1=MergeSketch.estimate()
    MergeSketch.update(ZeroSketch.Array())
    f1pure=MergeSketch.estimate()-ZeroSketch.estimate()
    return f1pure

In [4]:
file1rdd=sc.textFile('/user/poi/poi-50-1kww/poi-50-1kww')
file1rdd.first()

'444087624'

In [4]:
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01

count n time: 28.732884645462036
n: 1000032019


In [11]:
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)

Collect Time: 41.18265962600708


In [12]:
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

DCL1 time: 3.4090662002563477
d: 787267195.931828
f1: 605086523.0236659
DCL1: 1993420387.018037


In [13]:
file1rdd=sc.textFile('/user/poi/poi-50-5kww/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 77.85217380523682
n: 5000084437
Collect Time: 124.36372470855713
DCL1 time: 12.876952886581421
d: 3971129021.0032535
f1: 3104075342.728803
DCL1: 10472513278.144278


In [3]:
file1rdd=sc.textFile('/user/poi/poi-100-1kww/poi-100-1kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 29.20608401298523
n: 999945681
Collect Time: 46.57390737533569
DCL1 time: 3.209986686706543
d: 630873465.38572
f1: 362534116.03174114
DCL1: 989688972.7775296


In [4]:
file1rdd=sc.textFile('/user/poi/poi-100-5kww/poi-100-5kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 98.67343759536743
n: 5000013559
Collect Time: 141.27398657798767
DCL1 time: 13.502843379974365
d: 3193044247.025442
f1: 1881753362.694775
DCL1: 5119926986.378858


In [5]:
file1rdd=sc.textFile('/user/poi/poi-200-1kww/poi-200-1kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 27.6564838886261
n: 999956736
Collect Time: 47.4516818523407
DCL1 time: 2.8113203048706055
d: 431816663.14697224
f1: 131990409.25469714
DCL1: 497482411.155295


In [6]:
file1rdd=sc.textFile('/user/poi/poi-200-5kww/poi-200-5kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 99.46901178359985
n: 5000041249
Collect Time: 144.3686022758484
DCL1 time: 12.575514793395996
d: 2187156266.37084
f1: 695201508.3721356
DCL1: 2540366705.5601068


In [14]:
file1rdd=sc.textFile('/user/poi/poi-50-1www/poi-50-1www')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 154.92816495895386
n: 9999969413
Collect Time: 246.10641264915466
DCL1 time: 27.286706686019897
d: 7925433568.293607
f1: 6156811768.681699
DCL1: 20622129145.51352


In [15]:
file1rdd=sc.textFile('/user/zipf/zipf-2-1kww/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 23.224120140075684
n: 999590784
Collect Time: 43.67264199256897
DCL1 time: 3.1156017780303955
d: 982111665.846918
f1: 970160533.4956743
DCL1: 33357166630.13696


In [16]:
file1rdd=sc.textFile('/user/zipf/zipf-2-5kww/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 79.1268265247345
n: 4998045354
Collect Time: 125.66408252716064
DCL1 time: 16.92526340484619
d: 4932968018.070068
f1: 4846064639.790714
DCL1: 162225832484.20703


In [17]:
file1rdd=sc.textFile('/user/zipf/zipf-2-1www/')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 152.16044759750366
n: 9926629697
Collect Time: 243.78100538253784
DCL1 time: 27.1193585395813
d: 9819772637.430153
f1: 9730099133.778145
DCL1: 495990267785.801


In [7]:
file1rdd=sc.textFile('/user/zipf/zipf-1.2-1kww/zipf-1.2-1kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 23.322619676589966
n: 865596872
Collect Time: 36.94438457489014
DCL1 time: 2.771010160446167
d: 655250148.3287772
f1: 585514758.601017
DCL1: 2025057837.1027997


In [8]:
file1rdd=sc.textFile('/user/zipf/zipf-1.2-5kww/zipf-1.2-5kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 95.1595664024353
n: 4327773138
Collect Time: 135.31153464317322
DCL1 time: 10.545452117919922
d: 3306750588.4411817
f1: 2976831549.4372725
DCL1: 10593253247.867535


In [9]:
file1rdd=sc.textFile('/user/zipf/zipf-1.5-1kww/zipf-1.5-1kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 26.63609266281128
n: 989663491
Collect Time: 42.05757522583008
DCL1 time: 3.004051685333252
d: 910896606.9230926
f1: 868141318.729684
DCL1: 7418243922.946774


In [10]:
file1rdd=sc.textFile('/user/zipf/zipf-1.5-5kww/zipf-1.5-5kww')
begin=time.time()
n=file1rdd.count()
end=time.time()
print("count n time:",end-begin)
print("n:",n)
width=16
q=0.01
begin=time.time()
part_dict = file1rdd.mapPartitions(TransData)
a=part_dict.collect()
end=time.time()
print("Collect Time:",end-begin)
begin=time.time()
NDVhll=hll(width)
for i in a:
    NDVhll.update(i[1])
NDV=NDVhll.estimate()
csall=Countsketch(0.01,0.1)
for c in a:
    csall.update(c[2])
i2fi=csall.estimatef2()
Part_num=len(a)
for i in range(Part_num):
    del a[i][2]
b=sc.broadcast(pickle.dumps(a))
Aimrdd=sc.parallelize([i for i in range(Part_num)])
result=Aimrdd.mapPartitions(partComputef1).collect()
f1=sum(result)
DDCL1=DCL(NDV,i2fi,n,f1)
DCL1time=time.time()
print("DCL1 time:",DCL1time-begin)
print("d:",NDV)
print("f1:",f1)
print("DCL1:",DDCL1)

count n time: 98.50645399093628
n: 4948530006
Collect Time: 142.88439559936523
DCL1 time: 12.910451650619507
d: 4591601310.620675
f1: 4399408167.716371
DCL1: 41378206578.3719
